In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 1.1.0 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 1e-4
CUDA = torch.cuda.is_available()
PATH = 'soft-%dsteps-%dsamples' % (MCMC_SIZE, SAMPLE_SIZE)
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (enc_eta, enc_z)

In [ ]:
from ag_ep import *

train_prior(models, EUBO_init_eta_prior, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (102s),  symKL_DB_eta: 436.933,  symKL_DB_z: 545.433,  loss: -1739.091,  ess: 2.940,  kl_eta_ex: 1352.962,  kl_eta_in: 222.310,  kl_z_ex: 402.771,  kl_z_in: 41.329
epoch: 1\250 (107s),  symKL_DB_eta: 292.882,  symKL_DB_z: 295.346,  loss: -1374.128,  ess: 3.281,  kl_eta_ex: 801.393,  kl_eta_in: 205.828,  kl_z_ex: 201.177,  kl_z_in: 31.610
epoch: 2\250 (116s),  symKL_DB_eta: 213.515,  symKL_DB_z: 167.851,  loss: -1173.647,  ess: 3.557,  kl_eta_ex: 515.081,  kl_eta_in: 202.498,  kl_z_ex: 99.580,  kl_z_in: 23.954
epoch: 3\250 (116s),  symKL_DB_eta: 167.444,  symKL_DB_z: 110.060,  loss: -1059.778,  ess: 3.812,  kl_eta_ex: 379.192,  kl_eta_in: 213.740,  kl_z_ex: 58.290,  kl_z_in: 18.532
epoch: 4\250 (116s),  symKL_DB_eta: 141.116,  symKL_DB_z: 85.719,  loss: -988.941,  ess: 4.035,  kl_eta_ex: 313.263,  kl_eta_in: 238.470,  kl_z_ex: 44.044,  kl_z_in: 15.586
epoch: 5\250 (115s),  symKL_DB_eta: 127.562,  symKL_DB_z: 75.146,  loss: -941.669,  ess: 4.210,  kl_eta_ex: 282.725,  kl_eta

In [ ]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)